In [3]:
#vamos a usar swiss42
import tsplib95 #módulo de tsp, para problemas como el del agente viajero
import random  #nºs aleatorios
import math
from math import e #cte e
import copy #copia profunda

import urllib.request

In [4]:
file = 'swiss42.tsp'
#urllib.request.urlretrieve('http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz', file + '.gz')
##!gzip -d swiss.tsp.gz ##usamos extraer de zip (windows)


In [5]:
problem = tsplib95.load(file) #tsplib95.load_problem(file)

In [6]:
Nodos = list(problem.get_nodes())
Aristas = list(problem.get_edges())
print("nodos: " + str(len(Nodos)) + " aristas: " + str(len(Aristas)))
print(problem.get_weight(0, 1))

nodos: 42 aristas: 1764
15


In [7]:
dir(problem) # funciones, que podemos consultar en la doc

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_create_explicit_matrix',
 '_create_wfunc',
 '_defaults',
 '_special',
 '_wfunc',
 'as_dict',
 'as_keyword_dict',
 'as_name_dict',
 'comment',
 'dimension',
 'edge_weight_format',
 'edge_weight_type',
 'edge_weights',
 'fields_by_keyword',
 'fields_by_name',
 'get_display',
 'get_edges',
 'get_graph',
 'get_nodes',
 'get_weight',
 'is_complete',
 'is_depictable',
 'is_explicit',
 'is_full_matrix',
 'is_special',
 'is_symmetric',
 'is_weighted',
 'keywords_by_name',
 'load',
 'name',
 'names_by_keyword',
 'parse',
 'read',
 'render',
 'save',
 'special',
 'trace_canonical_tour',
 'trace_tours',
 'type',
 'validate',
 'wri

In [8]:
#la mejor solución al problema (de swiss42.tsp) es, casi seguro: 1273
def crear_solucion(Nodos):
    solucion = [Nodos[0]]
    for n in Nodos[1:]:
        solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
    return solucion

def distancia(a, b, problem):
    return problem.get_weight(a,b)

def distancia_total(solucion, problem):
    distancia_total = 0
    for i in range(len(solucion)-1):
        distancia_total += distancia(solucion[i], solucion[i+1], problem)
    return distancia_total + distancia(solucion[len(solucion)-1], solucion[0], problem)

In [9]:
distancia_total(crear_solucion(Nodos), problem)

4790

In [10]:
#búsqueda aleatoria
def busqueda_aleatoria(problem, N):
    Nodos = list(problem.get_nodes())
    mejor_solucion = []
    mejor_distancia = float('inf')
    
    for i in range(N):
        solucion = crear_solucion(Nodos)
        distancia = distancia_total(solucion, problem)
        if distancia < mejor_distancia:
            mejor_solucion = solucion
            mejor_distancia = distancia
            
    
    print("MEJOR SOLUCIÓN: ", mejor_solucion)
    print("DISTANCIA: ", mejor_distancia)
    return mejor_solucion

solucion = busqueda_aleatoria(problem, 5000)
        

MEJOR SOLUCIÓN:  [0, 8, 41, 23, 24, 21, 14, 16, 18, 9, 29, 22, 20, 12, 25, 11, 28, 26, 27, 39, 38, 30, 13, 33, 34, 32, 15, 19, 17, 1, 3, 10, 6, 37, 40, 2, 7, 36, 5, 4, 35, 31]
DISTANCIA:  3688


In [11]:
#búsqueda local
def genera_vecina(solucion):
    mejor_solucion = []
    mejor_distancia = 10e100
    for i in range(1, len(solucion)-1):
        for j in range(i+1, len(solucion)):
            #se genera una nueva solución intercambiando los 2 nodos
            #i,j usamos el operador + que concatena listas en python:
            vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
            #se evalúa la nueva solución
            distancia_vecina = distancia_total(vecina, problem)
            #y se guarda si mejora las anteriores
            if distancia_vecina <= mejor_distancia:
                mejor_distancia = distancia_vecina
                mejor_solucion = vecina
    return mejor_solucion

In [12]:
#a partir de la solución aleatoria anterior, hago esta mejora:
print("Distancia solución inicial: ", distancia_total(solucion, problem))

nueva_solucion = genera_vecina(solucion)
print("Distancia solución local:", distancia_total(nueva_solucion, problem))

Distancia solución inicial:  3688
Distancia solución local: 3431


In [13]:
def busqueda_local(problem):
    mejor_solucion = []
    #genero sol inicial
    solucion_referencia = crear_solucion(Nodos)
    mejor_distancia = distancia_total(solucion_referencia, problem)
    
    iteracion = 0
    while(1):
        iteracion+=1
        vecina = genera_vecina(solucion_referencia)
        distancia_vecina = distancia_total(vecina, problem)
        if(distancia_vecina < mejor_distancia):
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
        else:
            print("en la it", iteracion, ", la mejor solución encontrada es: ", mejor_solucion)
            print("Distancia :", mejor_distancia)
            return mejor_solucion
        solucion_referencia = vecina
        
sol = busqueda_local(problem)
            

en la it 41 , la mejor solución encontrada es:  [0, 22, 39, 9, 8, 4, 6, 5, 19, 13, 26, 18, 12, 11, 25, 10, 29, 30, 34, 32, 1, 7, 35, 36, 37, 15, 16, 14, 17, 31, 20, 33, 38, 24, 40, 21, 23, 41, 28, 2, 27, 3]
Distancia : 1812


In [ ]:
## https://www.sciencedirect.com/science/article/abs/pii/S0377221705008507
## https://www.researchgate.net/publication/222562743_A_simple_and_effective_iterated_greedy_algorithm_for_the_permutation_flowshop_scheduling_problem
## 